In [1]:
import numpy as np
import h5py

hf_x = h5py.File('../data/logspec_x.h5', 'r')
x_train = hf_x.get('x_train_data')[()]

hf_y = h5py.File('../data/logspec_y.h5', 'r')
y_train = hf_y.get('y_train_data')[()]

(x_train.shape, y_train.shape)

((64727, 99, 81, 1), (64727, 12))

In [2]:
from tensorflow.keras.layers import GRU, GlobalMaxPooling2D, Convolution2D, Dense, Input, Flatten, Dropout, MaxPooling2D, BatchNormalization, Conv3D, ConvLSTM2D
from tensorflow.keras.models import Sequential, Model
from sklearn.model_selection import GroupKFold
from sklearn.model_selection import train_test_split

In [5]:
input_shape = (99, 81, 1)
nclass = 12
inp = Input(shape=input_shape)
norm_inp = BatchNormalization()(inp)
img_1 = Convolution2D(8, kernel_size=2, activation='relu')(norm_inp)
img_1 = Convolution2D(8, kernel_size=2, activation='relu')(img_1)
img_1 = MaxPooling2D(pool_size=(2, 2))(img_1)
img_1 = Dropout(rate=0.2)(img_1)
img_1 = Convolution2D(16, kernel_size=3, activation='relu')(img_1)
img_1 = Convolution2D(16, kernel_size=3, activation='relu')(img_1)
img_1 = MaxPooling2D(pool_size=(2, 2))(img_1)
img_1 = Dropout(rate=0.2)(img_1)
img_1 = Convolution2D(32, kernel_size=3, activation='relu')(img_1)
img_1 = MaxPooling2D(pool_size=(2, 2))(img_1)
img_1 = Dropout(rate=0.2)(img_1)
img_1 = Flatten()(img_1)

dense_1 = BatchNormalization()(Dense(128, activation='relu')(img_1))
dense_1 = BatchNormalization()(Dense(128, activation='relu')(dense_1))
dense_1 = Dense(nclass, activation='softmax')(dense_1)

cnn = Model(inputs=inp, outputs=dense_1)
#opt = optimizers.Adam()

cnn.compile(optimizer='adam',loss='categorical_crossentropy',metrics=['categorical_accuracy'])
cnn.summary()

x_train, x_valid, y_train, y_valid = train_test_split(x_train, y_train, test_size=0.1, random_state=2017)
cnn.fit(x_train, y_train, batch_size=16, validation_data=(x_valid, y_valid), epochs=10, shuffle=True, verbose=1)

Model: "model_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_3 (InputLayer)         [(None, 99, 81, 1)]       0         
_________________________________________________________________
batch_normalization_v2_6 (Ba (None, 99, 81, 1)         4         
_________________________________________________________________
conv2d_10 (Conv2D)           (None, 98, 80, 8)         40        
_________________________________________________________________
conv2d_11 (Conv2D)           (None, 97, 79, 8)         264       
_________________________________________________________________
max_pooling2d_6 (MaxPooling2 (None, 48, 39, 8)         0         
_________________________________________________________________
dropout_6 (Dropout)          (None, 48, 39, 8)         0         
_________________________________________________________________
conv2d_12 (Conv2D)           (None, 46, 37, 16)        1168